In [3]:
import duckdb

In [4]:
con = duckdb.connect(database='dados_db.db', read_only=False)

In [5]:
tables = con.execute("SHOW TABLES").fetchall()
tables

[('bronze_produtos',)]

In [6]:
data = con.execute("SELECT * FROM bronze_produtos").fetchdf()
data.head()

,NATBR,MAKTX,WERKS,MAINS,LABST,DATA_INGESTAO,NOME_ARQUIVO
0,10001,PARAFUSO,BT10,100,100,2025-03-20 14:28:55.968631,z009_01.csv
1,10002,MARTELO,BT50,100,1500,2025-03-20 14:28:55.968631,z009_01.csv
2,10003,PREGO,BT10,100,50,2025-03-20 14:28:55.968631,z009_01.csv
3,10004,SERRA,BT50,100,200,2025-03-20 14:29:06.866122,z009_02.csv
4,10005,MACHADO,BT50,100,100,2025-03-20 14:29:06.866122,z009_02.csv


In [7]:
df = con.execute("""
                 SELECT * , ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY data_ingestao DESC) as row_num
                 FROM bronze_produtos
                 WHERE data_ingestao >= '2025-01-01'
                """).fetchdf()
df.head()

,NATBR,MAKTX,WERKS,MAINS,LABST,DATA_INGESTAO,NOME_ARQUIVO,row_num
0,10001,PARAFUSO,BT10,100,100,2025-03-20 14:28:55.968631,z009_01.csv,1
1,10004,SERRA,BT50,100,200,2025-03-20 14:29:06.866122,z009_02.csv,1
2,10003,PREGO,BT10,100,60,2025-03-20 14:29:06.866122,z009_02.csv,1
3,10003,PREGO,BT10,100,50,2025-03-20 14:28:55.968631,z009_01.csv,2
4,10002,MARTELO,BT50,100,1500,2025-03-20 14:28:55.968631,z009_01.csv,1


In [8]:
df_final = df.drop(columns=['NOME_ARQUIVO', 'DATA_INGESTAO', 'row_num'])
df_final = df_final.rename(columns={'NATBR': 'id',\
                                    'MAKTX': 'nm_produto',\
                                    'WERKS':'id_categoria',\
                                    'MAINS':'id_fornecedor',\
                                     'LABST':'vl_preco'})
df_final.head()

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10001,PARAFUSO,BT10,100,100
1,10004,SERRA,BT50,100,200
2,10003,PREGO,BT10,100,60
3,10003,PREGO,BT10,100,50
4,10002,MARTELO,BT50,100,1500


In [10]:
df_final.dtypes

id               object
nm_produto       object
id_categoria     object
id_fornecedor    object
vl_preco         object
dtype: object

In [16]:
df2 = df_final
df2 = df2.astype(
    {'id': int, 
     'nm_produto': str,
     'id_categoria': str, 
     'id_fornecedor': int, 
     'vl_preco': float})
df2.head()

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10001,PARAFUSO,BT10,100,100.0
1,10004,SERRA,BT50,100,200.0
2,10003,PREGO,BT10,100,60.0
3,10003,PREGO,BT10,100,50.0
4,10002,MARTELO,BT50,100,1500.0


In [17]:
df2.dtypes

id                 int64
nm_produto        object
id_categoria      object
id_fornecedor      int64
vl_preco         float64
dtype: object

In [21]:
con.execute("CREATE TABLE IF NOT EXISTS produtos "
"(id BIGINT, "
"nm_produto TEXT, "
"id_categoria TEXT, "
"id_fornecedor BIGINT, "
"vl_preco FLOAT)")

In [23]:
con.execute("INSERT INTO produtos Select * FROM df2")

In [24]:
df_result = con.execute("SELECT * FROM produtos").fetchdf()
df_result.head()

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10001,PARAFUSO,BT10,100,100.0
1,10004,SERRA,BT50,100,200.0
2,10003,PREGO,BT10,100,60.0
3,10003,PREGO,BT10,100,50.0
4,10002,MARTELO,BT50,100,1500.0


In [25]:
con.close()